In [ ]:
import os
root = os.path.abspath(os.path.join(os.getcwd(),'..'))

In [ ]:
root

In [ ]:
import geopandas as gpd
from shapely import geometry

In [ ]:
coast = gpd.read_file(os.path.join(root,'notebooks','ne_10m_coastline.shp'))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
coastline = coast.loc[coast.intersects(c.buffer(0.2))]#['geometry'].iloc[0]

In [ ]:
coastline.plot()

In [ ]:
island = geometry.Polygon(coastline.unary_union)

### Hexes option

In [ ]:
c = geometry.Point(107.93,-2.94)
center_hex = _flat_hex_polygon(c, 0.14)
big_center_hex = _flat_hex_polygon(c, .8)
c_ords = {ii:list(center_hex.exterior.coords)[ii] for ii in range(len(center_hex.exterior.coords))}
bc_ords = {ii:list(big_center_hex.exterior.coords)[ii] for ii in range(len(big_center_hex.exterior.coords))}
bonus_pt = geometry.Point(107.9,-3.2)
bpt = (bonus_pt.x, bonus_pt.y)
exteriors = []
for ii in range(6):
    if ii==3:
        exteriors.append(geometry.Polygon([c_ords[ii],bc_ords[ii],bc_ords[ii+1],bpt,c_ords[ii+1],c_ords[ii]]))
    elif ii==4:
        exteriors.append(geometry.Polygon([c_ords[ii],bpt,bc_ords[ii],bc_ords[ii+1],c_ords[ii+1],c_ords[ii]]))
    else:
        exteriors.append(geometry.Polygon([c_ords[ii],bc_ords[ii],bc_ords[ii+1],c_ords[ii+1],c_ords[ii]]))

In [ ]:
gdf = gpd.GeoSeries(exteriors+[center_hex])

In [ ]:
gdf = gdf.intersection(island)

In [ ]:
gdf.boundary.plot()

### GeoVoronoi option

In [ ]:
from geovoronoi import voronoi_regions_from_coords #polygon_lines_from_voronoi,polygon_shapes_from_voronoi_lines, coords_to_points, assign_points_to_voronoi_polygons
from scipy.spatial import Voronoi
import numpy as np
import pandas as pd

In [ ]:
def _resample_coords(coords, dist_thresh=0.014):
    """ increase the sampling of a coordinate sequence"""
    
    new_coords = []
    new_coords.append(coords[0])
    for ii_c in range(1,len(coords)):
        dist= np.linalg.norm(np.array(coords[ii_c])-np.array(new_coords[-1]))
        while dist>dist_thresh:

            n = np.array(coords[ii_c])-np.array(new_coords[-1])
            n /= np.linalg.norm(n) #unit vector
            new_coord = np.array(new_coords[-1])+n * 0.01
            new_coords.append(new_coord)
            dist= np.linalg.norm(np.array(coords[ii_c])-np.array(new_coord))

        new_coords.append(coords[ii_c])
    
    return new_coords

In [ ]:
bbox_buffer = geometry.box(*island.bounds).buffer(0.3)

In [ ]:
shape_coords = _resample_coords(island.exterior.coords)

In [ ]:
island_resampled = geometry.Polygon(shape_coords)

In [ ]:
shape_coords = np.array(shape_coords)

In [ ]:
region_polys, region_pts = voronoi_regions_from_coords(shape_coords, bbox_buffer)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,16))
gpd.GeoSeries(region_polys.values()).boundary.plot(ax=ax)
gpd.GeoSeries(island).boundary.plot(ax=ax, color='g')

In [ ]:
coverage = hexagon_coverage(geometry.box(*island.bounds).centroid,0.1)

In [ ]:
gpd.GeoSeries(coverage).boundary.plot(color='g')

In [ ]:
box = geometry.box(*island.bounds)

In [ ]:
r = 0.01

In [ ]:
def hcp(r, box):
    del_row = 2*r*np.cos(np.pi/6)
    del_col = 2*r
    shift = 2*r*np.sin(np.pi/6)
    xx,yy = np.meshgrid(
        np.arange(box.bounds[0], box.bounds[2], del_col),
        np.arange(box.bounds[1], box.bounds[3], del_row),
    )
    #print (xx.shape)
    
    shift_rows = np.arange(0,xx.shape[0],2).tolist()
    xx[tuple(shift_rows),:]+=shift
    
    xx = xx.flatten()
    yy = yy.flatten()
    
    return list(zip(xx.tolist(), yy.tolist()))
    

In [ ]:
pts = hcp(r, box)

In [ ]:
plt.scatter(np.array(pts)[:,0], np.array(pts)[:,1])

In [ ]:
multipt = geometry.MultiPoint(pts)

In [ ]:
valid_pts = multipt.intersection(island)

In [ ]:
valid_pts

In [ ]:
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
pt_array = np.array([(pt.x, pt.y) for pt in valid_pts.geoms])

In [ ]:
db = KMeans(n_clusters=2).fit(pt_array)

In [ ]:
db.labels_

In [ ]:
gdf = gpd.GeoDataFrame([gpd.GeoSeries(valid_pts.geoms), db.labels_]).T

In [ ]:
gdf = gdf.rename(columns={0:'geometry',1:'cluster'}).set_geometry('geometry')
gdf.plot('cluster')

In [ ]:
region_polys, region_pts = voronoi_regions_from_coords(pt_array, island_resampled)

In [ ]:
gdf_vor = gpd.GeoDataFrame(gpd.GeoSeries(region_polys.values())).rename(columns={0:'vor_geom'}).set_geometry('vor_geom')

In [ ]:
len(valid_pts)

In [ ]:
len(gdf_vor)

In [ ]:
gdf_vor.boundary.plot()

In [ ]:
import area

In [ ]:
gdf_vor = gpd.sjoin(gdf_vor, gdf)
gdf_vor.plot(column='cluster')

In [ ]:
gdf_vor.dissolve('cluster').plot()

In [ ]:
gdf_vor['area'] = gdf_vor['vor_geom'].apply(lambda geom: area.area(geometry.mapping(geom)))

In [ ]:
gdf_vor['area'].mean()/1000/1000

In [ ]:
gdf_vor['area'].sum()/1000/1000

In [ ]:
100000/270/5

In [ ]:
lc = gpd.read_file(os.path.join(root,'notebooks','id_lc.geojson')).explode('geometry')

In [ ]:
lc.boundary.plot()

In [ ]:
#lc.explode('geometry').apply(lambda r: r['geometry'].is_valid, axis=1)

In [ ]:
priority =["builtup", "plantation", "forest", "barren", "agriculture"]

In [ ]:
classes = lc.lc.unique()

In [ ]:
lc.loc[lc.lc=='builtup','geometry']=lc.loc[lc.lc=='builtup'].difference(lc.loc[lc.lc=='forest'].unary_union)

In [ ]:
for ii, cl1 in enumerate(priority):
    for cl2 in priority[ii+1:]:
        print (cl1,cl2)
        #print (lc.loc[cl2,'geometry'].difference(lc.loc[cl1,'geometry']))
        #_geom = lc.loc[cl2,'geometry'].difference(lc.loc[cl1,'geometry']).buffer(0)
        #print (lc.loc[cl2,'geometry'])
        lc.loc[lc.lc==cl2,'geometry'] = lc.loc[lc.lc==cl2].difference(lc.loc[lc.lc==cl1].unary_union)

In [ ]:
lc.boundary.plot()

In [ ]:
classes

In [ ]:
for col in classes:
    gdf_vor[col+'_area'] = 0 

In [ ]:
for idx, row in lc.iterrows():
    
    gdf_vor['intersection'] = gdf_vor.intersection(row['geometry'])
    gdf_vor['int_area'] = gdf_vor['intersection'].apply(lambda geom: area.area(geometry.mapping(geom)))
    gdf_vor[row['lc']+'_area'] += gdf_vor['int_area']

In [ ]:
classes

In [ ]:
gdf_vor[[col+'_area' for col in classes]].sum(1)

In [ ]:
gdf_vor['sum_area'] = gdf_vor[[col+'_area' for col in classes]].sum(1)

In [ ]:
gdf_vor[col+'_area'] = gdf_vor[col+'_area']/gdf_vor['sum_area']*gdf_vor['area']

In [ ]:
gdf_vor = gdf_vor.drop(columns=['intersection','int_area'])

In [ ]:
gdf_vor['lc'] = gdf_vor[[col+'_area' for col in classes]].idxmax(1)

In [ ]:
gdf_vor.plot('lc')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,16))
gdf_vor.boundary.plot(ax=ax, color='w',lw=1)
gdf_vor.plot('lc', ax=ax, alpha=0.5)
gdf_vor.dissolve('cluster').boundary.plot(ax=ax, color='#575757', lw=3)

In [ ]:
for col in gdf_vor.columns:
    if 'area' in col:
        gdf_vor[col] /= 1e6

In [ ]:
gdf_vor

In [ ]:
gdf_vor = gdf_vor.drop(columns=['index_right','lc','area']).rename(columns={'sum_area':'area','vor_geom':'geometry'})

In [ ]:
gdf_vor['node_id'] = gdf['cluster'].map({0:'NODE_A',1:'NODE_B'})

In [ ]:
gdf_vor['solar_irradiance'] = 800+(1400-800)*(gdf_vor.geometry.centroid.y - gdf_vor.geometry.centroid.y.min())/(gdf_vor.geometry.centroid.y.max()-gdf_vor.geometry.centroid.y.min())

In [ ]:
gdf_vor['population'] = gdf_vor['builtup_area']*1210

In [ ]:
gdf_vor

In [ ]:
gdf_vor.set_geometry('geometry')

In [ ]:
gdf_vor.drop(columns=['cluster']).set_geometry('geometry').to_file(os.path.join(root,'gridland_2node.geojson'), driver='GeoJSON')

In [ ]:
a = list(range(3,6))

In [ ]:
a[::-1]

- carbon constraint
- final energy demand
- final energy efficiency
- population growth
- ag growth
- plantation growth
- pop density
- Coal price
- Technologies:
  - Generation
    - utility solar {construction period, cost, }
    - distributed solar {construction period, cost, land_cost}
    - coal
  - Transmission
    - BIG
    - SMALL
    - local {loss per km}

In [ ]:
from shapely import geometry

In [ ]:
from shapely.geometry import Polygon
from shapely.affinity import translate
import math
from collections import namedtuple


Hexagons = namedtuple(
    "Hexagons",
    ["centre", "top", "topright", "bottomright", "bottom", "bottomleft", "topleft"],
)


def _flat_hex_coords(centre, size, i):
    """Return the point coordinate of a flat-topped regular hexagon.
    points are returned in counter-clockwise order as i increases
    the first coordinate (i=0) will be:
    centre.x + size, centre.y
    """
    angle_deg = 60 * i
    angle_rad = math.pi / 180 * angle_deg
    return (
        centre.x + size * math.cos(angle_rad),
        centre.y + size * math.sin(angle_rad),
    )


def _flat_hex_polygon(centre, size):
    """Return a flat-topped regular hexagonal Polygon, given a centroid Point and side length"""
    return Polygon([_flat_hex_coords(centre, size, i) for i in range(6)])


def hexagon_coverage(centre, size):
    """Tile an area having a Shapely Point centroid _centre_ with regular flat-topped
    hexagonal polygons having side-length _size_.
    The returned namedtuple has seven entries:
    0 is the central polygon, 1 - 6 are surrounding polygons, beginning directly
    above 0, progressing clockwise.
    """
    cp = _flat_hex_polygon(centre, size)
    width = 2 * size
    height = math.sqrt(3) * size
    horizontal_distance = width * 0.75
    vertical_distance = height * 0.5
    # second hex is directly above central_polygon, progresses clockwise
    return Hexagons(
        cp,
        translate(cp, 0, vertical_distance * 2),
        translate(cp, horizontal_distance, vertical_distance),
        translate(cp, horizontal_distance, -vertical_distance),
        translate(cp, 0, vertical_distance * -2),
        translate(cp, -horizontal_distance, vertical_distance),
        translate(cp, -horizontal_distance, -vertical_distance),
    )
